In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('normalization_assesment_dataset_10k.csv')
print("Dataset shape: ",df.shape)
df.head()

In [ ]:
def handle_na(df):    
    # Calculate null values for each column
    null_counts = df.isnull().sum()
    # Calculate percentage of null values
    null_percentages = (null_counts / len(df)) * 100
    print('null_percentages',null_percentages)
    if all(null_percentages[column] for column in df.columns)  and df.shape[0]>=10000:
        new_df = df.dropna()
        print(new_df.shape)
        return new_df
    return df

df = handle_na(df)
df.head()

In [ ]:
max_char=0
max_words=0
for row in df['raw_comp_writers_text']:
    if len(str(row).split(' ')) > max_words:
        max_words = len(str(row).split(' '))
    if len(row) > max_char:
       max_char=len(row)
print(max_char)
print(max_words)
    
    

In [ ]:
import matplotlib.pyplot as plt

def plot_word_counts(df_column):
   word_counts = df_column.str.split().str.len()
   
   plt.figure(figsize=(10, 6))
   plt.hist(word_counts, bins=20, edgecolor='black')
   plt.title('Distribution of Words per Row')
   plt.xlabel('Number of Words')
   plt.ylabel('Number of rows')
   plt.grid(True, alpha=0.3)
   plt.show()
   
   print(f"Average words per row: {word_counts.mean():.1f}")
   print(f"Max words: {word_counts.max()}")

plot_word_counts(df['raw_comp_writers_text'])

In [ ]:
raw_column='raw_comp_writers_text'

In [ ]:
import re
def find_non_alpha(df, column_name):
    # Combine all strings in column
    text = ' '.join(df[column_name].astype(str))
    # Find all non-alphabetical characters
    non_alpha = re.findall(r'[^a-zA-Z\s]', text)
    # Count occurrences
    char_counts = {char: text.count(char) for char in set(non_alpha)}
    return char_counts

In [ ]:
from pprint import pprint
non_alpha_chars = find_non_alpha(df, raw_column)
print("Non-alphabetical characters found:")
print(non_alpha_chars)
print(len(non_alpha_chars))

In [ ]:
def find_punctuation_examples(df, column_name):
   """Print one row per punctuation mark found"""
   import string

   shown_examples = set()
   
   for idx, row in df.iterrows():
       text = str(row[column_name])
       for char in text:
           if char in string.punctuation and char not in shown_examples:
               print(f"\nPunctuation '{char}' in row {idx}:")
               print(row)
               shown_examples.add(char)

punct_marks = find_punctuation_examples(df, 'raw_comp_writers_text')

In [ ]:
import string
def find_non_letter_examples(df, column_name):
    """Print one row per non-letter character found"""
    
    shown_examples = set()
    
    for idx, row in df.iterrows():
        text = str(row[column_name])
        for char in text:
            # Check if character is not a letter and not whitespace
            if not char in string.ascii_letters and not char.isspace() and char not in shown_examples:
                print(f"\nNon-letter character '{char}' in row {idx}:")
                print(row)
                shown_examples.add(char)

non_letter_chars = find_non_letter_examples(df, raw_column)

In [ ]:
def find_angled_brackets_content(df, column_name):
    """Extract strings between < > characters in a dataframe column"""
    import re
    
    bracketed_content = set()
    
    for idx, row in df.iterrows():
        text = str(row[column_name])
        # Find all matches between < >
        matches = re.findall(r'<([^>]+)>', text)
        if matches:
            bracketed_content.update(matches)
    
    return list(bracketed_content)


brackets = find_angled_brackets_content(df, raw_column)
print("\nUnique bracketed content found:", brackets)


In [ ]:
def heuristic_clean(raw_text, keywords):
   """Clean text using heuristic rules"""
   import string
   
   # Remove keywords
   clean_text = raw_text
   for keyword in keywords:
       clean_text = clean_text.replace(keyword, '')
   
   # Replace & and comma with /
   clean_text = clean_text.replace(' & ', '/').replace('&', '/').replace(',', '/')
   
   # Replace punctuation except / with space
   trans = str.maketrans({p: ' ' for p in string.punctuation if p != '/'})
   clean_text = clean_text.translate(trans)
   
   # Replace multiple spaces with single space
   clean_text = ' '.join(clean_text.split())
   
   # Remove spaces around /
   clean_text = re.sub(r'\s*/\s*', '/', clean_text)

   return clean_text


text = '<Unknown>/Wright, Justyce Kaseem'
clean_text = heuristic_clean(text, brackets)
print(clean_text)

In [ ]:
keywords = [
    # Publishing and Rights Management
    'COPYRIGHT CONTROL', 'PUBLISHING', 'MUSIC PUBLISHING', 'ALL RIGHTS RESERVED',
    'RIGHTS ADMINISTERED', 'RIGHTS MANAGED', 'RIGHTS CONTROLLED',
    'PERFORMANCE RIGHTS', 'MECHANICAL RIGHTS', 'SYNC RIGHTS',
    'ADMINISTERED BY', 'LICENSED TO', 'CONTROLLED BY',
    
    # Business Entities
    'LIMITED', 'LTD', 'LLC', 'INC', 'INCORPORATED', 'CORP', 'CORPORATION',
    'GMBH', 'PTY', 'S.A.', 'N.V.', 'AG', 'CO', 'COMPANY',
    
    # Music Industry Terms
    'MUSIC', 'SONGS', 'PRODUCTIONS', 'ENTERTAINMENT', 'RECORDS',
    'RECORDINGS', 'LABEL', 'STUDIO', 'GROUP', 'BAND',
    
    # Major Companies and Common Affiliates
    'SONY', 'ATV', 'SONY/ATV', 'BMG', 'EMI', 'UNIVERSAL',
    'WARNER', 'WARNER CHAPPELL', 'KOBALT', 'BMI', 'ASCAP', 'SESAC',
    'COLUMBIA', 'ATLANTIC', 'CAPITOL', 'MOTOWN', 'RCA',
    
    # Professional Designations
    'MUSIKVERLAG', 'ÉDITIONS', 'EDITIONS', 'VERLAG',
    'MUSIC GROUP', 'MEDIA', 'PARTNERS', 'ASSOCIATES',
    
    # Common Suffixes and Descriptors
    'WORLDWIDE', 'INTERNATIONAL', 'GLOBAL', 'MANAGEMENT',
    'HOLDINGS', 'VENTURES', 'ENTERPRISES', 'WORKS',
    
    # Rights Organizations
    'PERFORMING RIGHTS', 'SOCIETY', 'ORGANIZATION', 'ASSOCIATION',
    'COLLECTION SOCIETY', 'RIGHTS SOCIETY',
    
    # Digital and Modern Terms
    'DIGITAL', 'DISTRIBUTION', 'STREAMING', 'LICENSING',
    
    # Legal and Administrative
    'ADMINISTERED', 'REPRESENTS', 'REPRESENTED BY',
    'ON BEHALF OF', 'C/O', 'CARE OF',
    
    # Geographical Indicators
    '(UK)', '(US)', '(EU)', '(JP)', 'UK', 'USA', 'AMERICA',
    'EUROPEAN', 'INTERNATIONAL', 'GLOBAL'
]

# Clean up the keywords list (remove duplicates, strip whitespace)
keywords = list(set([k.strip() for k in keywords]))
# Sort by length (longer phrases first to avoid partial replacements)
keywords = sorted(keywords, key=len, reverse=True)
print(keywords)
keywords=keywords+brackets

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Split data
X = df['raw_comp_writers_text']
y = df['CLEAN_TEXT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Make predictions using heuristic_clean
y_pred = [heuristic_clean(text, keywords) for text in X_test]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')

# Show some examples
print('\nExample predictions:')
for i in range(5):
   print(f'\nInput: {X_test.iloc[i]}')
   print(f'Predicted: {y_pred[i]}')
   print(f'Expected: {y_test.iloc[i]}')